# 22F-3853 Muhammad Abu Huraira  
# 22F-3410 Shayan Zawar   
# NLP A02


# Read file and Seperate senetences from it

In [69]:
import pandas as pd
file=pd.read_csv('/kaggle/input/urdu-dataset-20000/final_main_dataset.tsv',sep='\t')
file
print("Export only sentences to newer file")
sentences = file['sentence']
sentences
sentences.to_csv('urdu_sentences.txt', index=False, header=False)

Export only sentences to newer file


# Tokenize the txt file

In [70]:
!pip install sentencepiece


In [71]:
import sentencepiece as spm
import os

def train_sentencepiece_tokenizers():    
    print(" Training SentencePiece Tokenizers with BPE...")
    
    
    urdu_file = "/kaggle/working/urdu_sentences.txt"
    
    urdu_model_prefix = "/kaggle/working/urdu_tokenizer"
    
    # Training parameters for ~10K vocabulary
    vocab_size = 10000
    
    # Train Urdu tokenizer
    print("\n Training Urdu SentencePiece tokenizer...")
    if os.path.exists(urdu_file):
        spm.SentencePieceTrainer.train(
            input=urdu_file,
            model_prefix=urdu_model_prefix,
            vocab_size=vocab_size,
            model_type='bpe',  # Use BPE algorithm
            character_coverage=0.995,  # Good for Urdu script
            split_by_unicode_script=True,  # Important for Urdu
            split_by_whitespace=True,
            normalization_rule_name='nfkc',  # Unicode normalization
            remove_extra_whitespaces=True,
            max_sentence_length=1000,
            shuffle_input_sentence=True,
            seed_sentencepiece_size=1000000,
            shrinking_factor=0.75,
            num_threads=4,
            train_extremely_large_corpus=False,
            # Special tokens
            pad_id=0,
            unk_id=1,
            bos_id=2,
            eos_id=3,
            pad_piece='<PAD>',
            unk_piece='<UNK>',
            bos_piece='<SOS>',
            eos_piece='<EOS>',
            user_defined_symbols=['<MASK>', '<extra_id_0>', '<extra_id_1>', '<extra_id_2>']
        )
        print(f" Urdu tokenizer trained: {urdu_model_prefix}.model")
        print(f"   Vocabulary file: {urdu_model_prefix}.vocab")
    else:
        print(f" Urdu file not found: {urdu_file}")
    
# Test the tokenizers
def test_tokenizers(urdu_model_path):
    """Test the trained tokenizers"""
    print("\n Testing trained tokenizers...")
    
    try:
        # Load tokenizers
        urdu_sp = spm.SentencePieceProcessor()
        urdu_sp.load(urdu_model_path)
        
        # Test sentences
        urdu_test = "تو کبھی خود کو بھی دیکھے گا تو ڈر جائے گا"
       
        
        print(f"\n Tokenizer Statistics:")
        print(f"Urdu vocabulary size: {urdu_sp.get_piece_size()}")
       
        
        print(f"\n Test Encoding/Decoding:")
        print(f"Original Urdu: {urdu_test}")
        urdu_encoded = urdu_sp.encode(urdu_test, out_type=int)
        urdu_decoded = urdu_sp.decode(urdu_encoded)
        print(f"Encoded: {urdu_encoded}")
        print(f"Decoded: {urdu_decoded}")
        
        
        print("\n Tokenizer testing completed!")
        
    except Exception as e:
        print(f" Error testing tokenizers: {e}")

# Train the tokenizers
print(" Starting SentencePiece BPE Training Process...")
urdu_model_path = train_sentencepiece_tokenizers()

# Test the trained tokenizers
test_tokenizers('/kaggle/working/urdu_tokenizer.model')

print("\n SentencePiece tokenizers with BPE successfully trained!")
print(" Model files created:")
print(f"   - Urdu: /kaggle/working/urdu_tokenizer.model")

 Starting SentencePiece BPE Training Process...
 Training SentencePiece Tokenizers with BPE...

 Training Urdu SentencePiece tokenizer...
 Urdu tokenizer trained: /kaggle/working/urdu_tokenizer.model
   Vocabulary file: /kaggle/working/urdu_tokenizer.vocab

 Testing trained tokenizers...

 Tokenizer Statistics:
Urdu vocabulary size: 10000

 Test Encoding/Decoding:
Original Urdu: تو کبھی خود کو بھی دیکھے گا تو ڈر جائے گا
Encoded: [68, 397, 490, 36, 76, 6097, 125, 68, 878, 280, 125]
Decoded: تو کبھی خود کو بھی دیکھے گا تو ڈر جائے گا

 Tokenizer testing completed!

 SentencePiece tokenizers with BPE successfully trained!
 Model files created:
   - Urdu: /kaggle/working/urdu_tokenizer.model


In [72]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time

from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [73]:
class EncoderDecoder(nn.Module):

    def __init__(self,encoder,decoder,src_embed,tgt_embed,generator):
        # calling inherited class constructor
        super(EncoderDecoder,self).__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.src_embed=src_embed
        self.tgt_embed=tgt_embed
        self.generator=generator


    # to process src to tgt
    def forward(self,src,tgt,src_mask,tgt_mask):
        return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)


    def encode(self,src,src_mask):
        return self.encoder(self.src_embed(src),src_mask)


    def decode(self,memory,src_mask,tgt,tgt_mask):
        return self.decoder(self.tgt_embed(tgt),memory,src_mask,tgt_mask)

In [74]:
# the purpose of this code is to now genrator or predict
# word after final layer using softmax
class Generator(nn.Module):

    def __init__(self,d_model,vocab):
        super(Generator,self).__init__()
        # the linear layer here maps final model vocab to get there probabilities
        self.proj=nn.Linear(d_model,vocab)

    def forward(self,x):
        return F.log_softmax(self.proj(x),dim=1)
        
        

# Encoder Decoder Code
here we will create encoder decoder code for N=2 layers for encoder and decoder

In [75]:
# code copies modeul N times!
def clones(module,N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [76]:
# entire encoder module consisting of N layers
class Encoder(nn.Module):
    def __init__(self,layer,N):
        super(Encoder,self).__init__()
        self.layers=clones(layer,N)
        self.norm=LayerNorm(layer.size)

  #"Here we are passing input and mask through each layer"
    def forward(self,x,mask):
        for layer in self.layers:
            x=layer(x,mask)
        return self.norm(x)

In [77]:
# add sublayers inputs 
class LayerNorm(nn.Module):

    # here a is scale and b is shift
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.a_2=nn.Parameter(torch.ones(features))
        self.b_2=nn.Parameter(torch.zeros(features))
        self.eps=eps

    def forward(self,x):
        mean=x.mean(-1,keepdim=True)
        std=x.std(-1,keepdim=True)
        return self.a_2*(x-mean)/(std + self.eps)+self.b_2

In [78]:
# normalization within each layer
class SublayerConnection(nn.Module):

    def __init__(self,size,dropout):
        super(SublayerConnection,self).__init__()
        self.norm=LayerNorm(size)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
# Each encoder in one layer have two part
# ie encoder has attention and feed forward network 

class EncoderLayer(nn.Module):
    def __init__(self,size,self_attn,feed_forward,dropout):
        super(EncoderLayer,self).__init__()
        self.self_attn=self_attn
        self.feed_forward=feed_forward
        self.sublayer=clones(SublayerConnection(size,dropout),2)
        self.size=size
    def forward(self,x,mask):
        x=self.sublayer[0](x,lambda x:self.self_attn(x,x,x,mask))
        return self.sublayer[1](x,self.feed_forward)

# Decoder

In [80]:
class Decoder(nn.Module):

    def __init__(self,layer,N):
        super(Decoder,self).__init__()
        self.layer=clones(layer,N)
        self.norm=LayerNorm(layer.size)
    def forward(self,x,memory,src_mask,tgt_mask):
        for layer in self.layer:
            x=layer(x,memory,src_mask,tgt_mask)
        return self.norm(x)

In [81]:
# unlike encoder that has only attn and feed forward
#decoder have two types of attn one for encoders ouput and feed
class DecoderLayer(nn.Module):
    def __init__(self,size,self_attn,src_attn,feed_forward,dropout):
        super(DecoderLayer,self).__init__()
        self.size=size
        self.self_attn=self_attn
        self.src_attn=src_attn
        self.feed_forward=feed_forward
        self.sublayer=clones(SublayerConnection(size,dropout),3)
        
    def forward(self,x,memory,src_mask,tgt_mask):
        m=memory
        x=self.sublayer[0](x, lambda x:self.self_attn(x,x,x,tgt_mask))
        x=self.sublayer[1](x,lambda x:self.src_attn(x,m,m,src_mask))
        return self.sublayer[2](x,self.feed_forward)        


In [ ]:
# this part of code mask the future 
# output just like traignle
# embedding/tokens to prevent cheating 
def subsequent_mask(size):
    attn_shape=(1,size,size)
    subsequent_mask=np.triu(np.ones(attn_shape),k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask)==0

# Attention

In [83]:
def attention(query,key,value,mask=None,dropout=None):
    d_k=query.size(-1)
    # to get smooth gradient 
    scores=torch.matmul(query,key.transpose(-2,-1))/ math.sqrt(d_k)

    if mask is not None:
        scores=scores.masked_fill(mask==0,-1e9)
    p_attn=F.softmax(scores,dim=-1)
    if dropout is not None:
        p_attn=dropout(p_attn)
    return torch.matmul(p_attn,value),p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention,self).__init__()
        assert d_model% h ==0
        self.d_k=d_model //h
        self.h=h
        self.linears=clones(nn.Linear(d_model,d_model),2)
        self.attn=None
        self.dropout=nn.Dropout(p=dropout)

    def forward(self,query,key,value,mask=None):
        if mask is not None:
            mask=mask.unsqueeze(1)
        nbatches=query.size(0)
        query,key,value=[l(x).view(nbatches,-1,self.h,self.d_k).transpose(1,2) for l,x in zip(self.linears,(query,key,value))]
        x,self.attn=attention(query,key,value,mask=mask,dropout=self.dropout)
    
        x=x.transpose(1,2).contiguous().view(nbatches,-1 ,self.h* self.d_k)

        return self.linears[-1](x)

#  Embedding

# Positional Encoding of Embeddings
As Unlike RNN Transformer does not have architecture that allows it to store seq of word to get this behaviour we use positional encoding of embeddings

In [85]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,d_ff,dropout=0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.w_1=nn.Linear(d_model,d_ff)
        self.w_2=nn.Linear(d_ff,d_model)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
        

In [86]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [87]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout,max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout=nn.Dropout(p=dropout)

        # calculating positional encoding
        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2)* -(math.log(10000.0)/d_model))
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0)
        self.register_buffer('pe',pe)

    def forward(self,x):
        x=x+Variable(self.pe[:,:x.size(1)],requires_grad=False)
        return self.dropout(x)

In [ ]:
def make_model(src_vocab, tgt_vocab, N=2,d_model=512, d_ff=2048, h=4, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

# Training

## Batches and Masking

In [90]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [91]:
from contextlib import nullcontext
from tqdm import tqdm

def run_epoch(data_loader, model, loss_compute, device, is_train=True, repeats=1, desc=None):
    total_tokens = 0
    total_loss = 0
    total_correct = 0
    
    if is_train:
        model.train()
    else:
        model.eval()

    base_desc = desc or ("Training" if is_train else "Evaluating")

    for repeat_idx in range(repeats):
        pass_desc = base_desc if repeats == 1 else f"{base_desc} pass {repeat_idx + 1}/{repeats}"
        iterator = tqdm(data_loader, desc=pass_desc, leave=False)
        context = nullcontext() if is_train else torch.no_grad()

        with context:
            for batch_idx, (src, tgt) in enumerate(iterator):
                src, tgt = src.to(device), tgt.to(device)
                batch = Batch(src, tgt, pad=0)

                out = model.forward(batch.src, batch.trg,
                                    batch.src_mask, batch.trg_mask)
                loss_val = loss_compute(out, batch.trg_y, batch.ntokens)

                logits = model.generator(out)
                predictions = logits.argmax(dim=-1)
                mask = (batch.trg_y != 0)
                correct = ((predictions == batch.trg_y) & mask).sum().item()
                
                batch_tokens = batch.ntokens.item()
                total_loss += loss_val * batch_tokens
                total_tokens += batch_tokens
                total_correct += correct
                
                if is_train and batch_idx % 50 == 0:
                    batch_acc = (correct / batch_tokens) * 100 if batch_tokens > 0 else 0
                    iterator.set_postfix({
                        "loss": f"{loss_val:.4f}",
                        "acc": f"{batch_acc:.2f}%"
                    })

    avg_loss = (total_loss / total_tokens) if total_tokens else 0.0
    avg_accuracy = (total_correct / total_tokens * 100) if total_tokens else 0.0
    return avg_loss, avg_accuracy


# Optimizers 

In [92]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

# Label Smoothing

In [93]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))


# Loss Computation

In [94]:
class SimpleLossCompute:
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        if self.opt is not None and torch.isfinite(loss):
            loss.backward()
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() if torch.isfinite(loss) else 0.0


# Preparing for Unsupervised Data

In [95]:
import random
import itertools

SENTINEL_PIECES = ("<extra_id_0>", "<extra_id_1>", "<extra_id_2>")

def get_sentinel_ids(sp, required=2):
    ids = []
    for piece in SENTINEL_PIECES:
        pid = sp.piece_to_id(piece)
        if pid != sp.unk_id():
            ids.append(pid)
    if len(ids) < required:
        raise ValueError(
            f"Tokenizer is missing at least {required} sentinel pieces ({SENTINEL_PIECES}); "
            "regenerate SentencePiece with them via --user_defined_symbols."
        )
    return ids

def t5_style_span_corruption(tokens, sentinel_ids, noise_density=0.15, mean_span_length=3.0):
    """T5-style span corruption."""
    num_tokens = len(tokens)
    num_to_corrupt = int(round(num_tokens * noise_density))
    
    if num_to_corrupt == 0:
        return tokens, [sentinel_ids[0]]

    # Calculate the number of spans
    num_spans = int(round(num_to_corrupt / mean_span_length))
    if num_spans == 0 and num_to_corrupt > 0:
        num_spans = 1
    
    # Select span start positions
    span_starts = sorted(random.sample(range(num_tokens), num_spans))
    
    # Determine span lengths
    span_lengths = [int(random.expovariate(1.0 / mean_span_length)) + 1 for _ in range(num_spans)]
    
    # Create spans
    spans = []
    used_indices = set()
    for start, length in zip(span_starts, span_lengths):
        if start in used_indices:
            continue
        
        end = start
        current_length = 0
        while end < num_tokens and current_length < length:
            if end not in used_indices:
                used_indices.add(end)
                current_length += 1
            end += 1
        
        if current_length > 0:
            spans.append((start, end))

    spans.sort()

    sentinel_cycle = itertools.cycle(sentinel_ids)
    
    enc_tokens, dec_tokens, cursor = [], [], 0
    for start, end in spans:
        sentinel = next(sentinel_cycle)
        enc_tokens.extend(tokens[cursor:start])
        enc_tokens.append(sentinel)
        
        dec_tokens.append(sentinel)
        dec_tokens.extend(tokens[start:end])
        cursor = end
        
    enc_tokens.extend(tokens[cursor:])
    return enc_tokens, dec_tokens

def replace_with_random_words(tokens, vocab_size, num_to_replace):
    #Replace tokens with random tokens from the vocabulary.
    if not num_to_replace:
        return tokens
    
    positions = sorted(random.sample(range(len(tokens)), num_to_replace))
    
    new_tokens = list(tokens)
    for pos in positions:
        new_tokens[pos] = random.randint(4, vocab_size - 1) # Avoid special tokens
        
    return new_tokens


# Training on Data 

In [96]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

class UrduDenoiseDataset(Dataset):
    def __init__(self, texts, sp, return_original=False):
        self.texts = texts
        self.sp = sp
        self.bos = sp.bos_id()
        self.eos = sp.eos_id()
        self.sentinel_ids = get_sentinel_ids(sp, required=2)
        self.vocab_size = len(sp)
        self.return_original = return_original
    
    def __getitem__(self, idx):
        original_text = self.texts[idx]
        original_tokens = self.sp.encode(original_text, out_type=int)
        
        enc_tokens, dec_tokens = t5_style_span_corruption(original_tokens, self.sentinel_ids)

        src = torch.tensor([self.bos] + enc_tokens + [self.eos])
        tgt = torch.tensor([self.bos] + dec_tokens + [self.eos])
        
        if self.return_original:
            return src, tgt, original_text
        return src, tgt
    
    def __len__(self):
        return len(self.texts)

In [97]:
from pathlib import Path
from torch.utils.data import random_split

def collate_fn(batch):
    if len(batch[0]) == 3:  # This is a batch for evaluation
        src_batch, tgt_batch, original_texts = zip(*batch)
        src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
        tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
        return src_batch, tgt_batch, original_texts
    else:  # This is a batch for training
        src_batch, tgt_batch = zip(*batch)
        src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
        tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
        return src_batch, tgt_batch

TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16

with open("/kaggle/working/urdu_sentences.txt", encoding="utf-8") as f:
    urdu_texts = [line.strip() for line in f.readlines() if line.strip()]

TOKENIZER_PATH = Path("/kaggle/working/urdu_tokenizer.model")

urdu_sp = spm.SentencePieceProcessor()
urdu_sp.load(str(TOKENIZER_PATH))


train_val_dataset = UrduDenoiseDataset(urdu_texts, urdu_sp, return_original=False)
test_dataset_eval = UrduDenoiseDataset(urdu_texts, urdu_sp, return_original=True)


# Split dataset: Train 80%, Validation 10%, Test 10%
train_size = int(0.8 * len(train_val_dataset))
val_size = int(0.1 * len(train_val_dataset))
test_size = len(train_val_dataset) - train_size - val_size

train_dataset, val_dataset, _ = random_split(train_val_dataset, [train_size, val_size, test_size])
_, _, test_dataset_eval = random_split(test_dataset_eval, [train_size, val_size, test_size])


print(f"Dataset split: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset_eval)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset_eval, batch_size=EVAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


Dataset split: Train=16035, Val=2004, Test=2005


# To use multi gpu for training

In [98]:
import torch
import torch.nn as nn

class MultiGPULossCompute:

    def __init__(self, generator, criterion, devices, optimizer=None, chunk_size=5):
        self.generator = generator
        self.criterion = criterion
        self.devices = devices
        self.optimizer = optimizer
        self.chunk_size = chunk_size

    def __call__(self, out, targets, normalize=1):
        # Scatter to multiple GPUs
        out_scatter = nn.parallel.scatter(out, target_gpus=self.devices)
        targets_scatter = nn.parallel.scatter(targets, target_gpus=self.devices)

        # Replicate modules on each GPU
        generator_replicas = nn.parallel.replicate(self.generator, devices=self.devices)
        criterion_replicas = nn.parallel.replicate(self.criterion, devices=self.devices)

        total_loss = 0.0

        # Process in chunks to save memory
        for i in range(0, out_scatter[0].size(1), self.chunk_size):
            out_chunks = [o[:, i:i+self.chunk_size].contiguous() for o in out_scatter]
            target_chunks = [t[:, i:i+self.chunk_size].contiguous() for t in targets_scatter]

            # Forward pass on each GPU
            gen_outputs = nn.parallel.parallel_apply(generator_replicas, out_chunks)

            # Compute loss on each GPU
            losses = nn.parallel.parallel_apply(criterion_replicas, list(zip(gen_outputs, target_chunks)))

            # Combine and backward
            loss_tensor = torch.stack([l.mean().to(self.devices[0]) for l in losses])
            loss = loss_tensor.mean()
            loss.backward()

            total_loss += loss.item()

        # Optimizer step
        if self.optimizer is not None:
            self.optimizer.step()
            self.optimizer.zero_grad()

        return total_loss / normalize


# Model Training Loops

# Phase 1: T5-Style Span Corruption Training

In this phase, we train the model using T5-style span corruption where random spans of text are masked and the model learns to predict the masked content. This approach is excellent for learning robust representations of the language.

In [ ]:
print("\n" + "="*80)
print("PHASE 1: TRAINING WITH T5-STYLE SPAN CORRUPTION")
print("="*80)

# Initialize model for span corruption training
model_t5 = make_model(len(urdu_sp), len(urdu_sp), N=2)
criterion = LabelSmoothing(size=len(urdu_sp), padding_idx=0, smoothing=0.1)
optimizer_t5 = get_std_opt(model_t5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5.to(device)

def _base_model(module):
    return module.module if isinstance(module, nn.DataParallel) else module

base_model_t5 = _base_model(model_t5)
generator_t5 = base_model_t5.generator

best_val_loss_t5 = float('inf')
num_epochs_t5 = 15

training_history_t5 = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print(f"\nTraining on device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model_t5.parameters()):,}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

for epoch in range(num_epochs_t5):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch + 1}/{num_epochs_t5}")
    print(f"{'='*80}")
    
    loss_compute_train = SimpleLossCompute(generator_t5, criterion, optimizer_t5)
    avg_train_loss, avg_train_acc = run_epoch(
        train_loader, model_t5, loss_compute_train, device, is_train=True, desc="T5 Span Corruption Training"
    )
    
    loss_compute_eval = SimpleLossCompute(generator_t5, criterion, opt=None)
    avg_val_loss, avg_val_acc = run_epoch(
        val_loader, model_t5, loss_compute_eval, device, is_train=False, desc="Validation"
    )
    
    # Store history
    training_history_t5['train_loss'].append(avg_train_loss)
    training_history_t5['train_acc'].append(avg_train_acc)
    training_history_t5['val_loss'].append(avg_val_loss)
    training_history_t5['val_acc'].append(avg_val_acc)
    
    print(f"\n📊 Epoch {epoch + 1} Results:")
    print(f"   Train Loss: {avg_train_loss:.4f} | Train Accuracy: {avg_train_acc:.2f}%")
    print(f"   Val Loss:   {avg_val_loss:.4f} | Val Accuracy:   {avg_val_acc:.2f}%")

    if avg_val_loss < best_val_loss_t5:
        best_val_loss_t5 = avg_val_loss
        torch.save(base_model_t5.state_dict(), "urdu_transformer_t5_best.pth")
        print(f"   ✓ Best T5 model saved (Val Loss: {best_val_loss_t5:.4f}, Val Acc: {avg_val_acc:.2f}%)")

# Save final T5 model
torch.save(base_model_t5.state_dict(), "urdu_transformer_t5_final.pth")

print("\n" + "="*80)
print("✅ T5-Style Span Corruption Training Completed!")
print(f"Best validation loss: {best_val_loss_t5:.4f}")
print("="*80)

# Load best model for next phase
model_t5.load_state_dict(torch.load("urdu_transformer_t5_best.pth"))
model_t5.to(device)

# Phase 2: Causal Language Modeling Training

Now we'll take the pre-trained model from T5-style training and fine-tune it using causal language modeling, where the model learns to predict the next token in a sequence. This is similar to GPT-style training.

In [ ]:
# Causal Language Modeling Dataset
class UrduCausalDataset(Dataset):
    """Dataset for causal language modeling where input and target are shifted versions."""
    def __init__(self, texts, sp, max_length=128):
        self.texts = texts
        self.sp = sp
        self.bos = sp.bos_id()
        self.eos = sp.eos_id()
        self.max_length = max_length
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        tokens = self.sp.encode(text, out_type=int)
        
        # Truncate if too long
        if len(tokens) > self.max_length - 2:
            tokens = tokens[:self.max_length - 2]
        
        # Add BOS and EOS
        full_tokens = [self.bos] + tokens + [self.eos]
        
        # For causal LM: input and target are the same sequence
        # The model will learn to predict next token
        src = torch.tensor(full_tokens)
        tgt = torch.tensor(full_tokens)
        
        return src, tgt
    
    def __len__(self):
        return len(self.texts)

# Create causal LM datasets
print("Creating Causal Language Modeling datasets...")
train_causal_dataset = UrduCausalDataset(urdu_texts, urdu_sp)
val_causal_dataset = UrduCausalDataset(urdu_texts, urdu_sp)
test_causal_dataset = UrduCausalDataset(urdu_texts, urdu_sp)

# Split the data
train_size = int(0.8 * len(train_causal_dataset))
val_size = int(0.1 * len(train_causal_dataset))
test_size = len(train_causal_dataset) - train_size - val_size

train_causal_dataset, val_causal_dataset, test_causal_dataset = random_split(
    train_causal_dataset, [train_size, val_size, test_size]
)

print(f"Causal LM Dataset split: Train={len(train_causal_dataset)}, Val={len(val_causal_dataset)}, Test={len(test_causal_dataset)}")

# Create dataloaders
def collate_fn_causal(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
    return src_batch, tgt_batch

train_causal_loader = DataLoader(train_causal_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_fn_causal)
val_causal_loader = DataLoader(val_causal_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn_causal)
test_causal_loader = DataLoader(test_causal_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn_causal)

print("✓ Causal LM DataLoaders created successfully!")

In [ ]:
print("\n" + "="*80)
print("PHASE 2: FINE-TUNING WITH CAUSAL LANGUAGE MODELING")
print("="*80)

# Initialize model from T5 pretrained weights
model_causal = make_model(len(urdu_sp), len(urdu_sp), N=2)
print("\n📥 Loading T5-pretrained weights...")
model_causal.load_state_dict(torch.load("urdu_transformer_t5_best.pth"))
model_causal.to(device)

# Create new optimizer for causal training
optimizer_causal = get_std_opt(model_causal)

base_model_causal = _base_model(model_causal)
generator_causal = base_model_causal.generator

best_val_loss_causal = float('inf')
num_epochs_causal = 10

training_history_causal = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print(f"\n✓ Starting causal LM fine-tuning...")
print(f"Training samples: {len(train_causal_dataset)}")
print(f"Validation samples: {len(val_causal_dataset)}")

for epoch in range(num_epochs_causal):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch + 1}/{num_epochs_causal}")
    print(f"{'='*80}")
    
    loss_compute_train = SimpleLossCompute(generator_causal, criterion, optimizer_causal)
    avg_train_loss, avg_train_acc = run_epoch(
        train_causal_loader, model_causal, loss_compute_train, device, is_train=True, desc="Causal LM Training"
    )
    
    loss_compute_eval = SimpleLossCompute(generator_causal, criterion, opt=None)
    avg_val_loss, avg_val_acc = run_epoch(
        val_causal_loader, model_causal, loss_compute_eval, device, is_train=False, desc="Validation"
    )
    
    # Store history
    training_history_causal['train_loss'].append(avg_train_loss)
    training_history_causal['train_acc'].append(avg_train_acc)
    training_history_causal['val_loss'].append(avg_val_loss)
    training_history_causal['val_acc'].append(avg_val_acc)
    
    print(f"\n📊 Epoch {epoch + 1} Results:")
    print(f"   Train Loss: {avg_train_loss:.4f} | Train Accuracy: {avg_train_acc:.2f}%")
    print(f"   Val Loss:   {avg_val_loss:.4f} | Val Accuracy:   {avg_val_acc:.2f}%")

    if avg_val_loss < best_val_loss_causal:
        best_val_loss_causal = avg_val_loss
        torch.save(base_model_causal.state_dict(), "urdu_transformer_causal_best.pth")
        print(f"   ✓ Best Causal model saved (Val Loss: {best_val_loss_causal:.4f}, Val Acc: {avg_val_acc:.2f}%)")

# Save final causal model
torch.save(base_model_causal.state_dict(), "urdu_transformer_causal_final.pth")

print("\n" + "="*80)
print("✅ Causal Language Modeling Fine-tuning Completed!")
print(f"Best validation loss: {best_val_loss_causal:.4f}")
print("="*80)

# Load best model for evaluation
model_causal.load_state_dict(torch.load("urdu_transformer_causal_best.pth"))
model_causal.to(device)

# Comprehensive Inference & Evaluation

Now we'll evaluate both models (T5 and Causal) with proper inference functions and comprehensive metrics.

In [ ]:
def generate_text_causal(model, sp, prompt, max_len=50, temperature=0.8, top_k=40, device='cuda'):
    """
    Generate text using causal language modeling (like GPT).
    The model predicts the next token given previous tokens.
    """
    model.eval()
    
    # Handle DataParallel
    if isinstance(model, nn.DataParallel):
        actual_model = model.module
    else:
        actual_model = model
    
    with torch.no_grad():
        # Encode the prompt
        tokens = sp.encode(prompt, out_type=int)
        src = torch.tensor([sp.bos_id()] + tokens, device=device).unsqueeze(0)
        
        # Create mask for source
        src_mask = (src != sp.pad_id()).unsqueeze(-2)
        
        # Encode the source
        memory = actual_model.encode(src, src_mask)
        
        # Initialize with BOS token
        ys = torch.tensor([[sp.bos_id()]], device=device)
        generated_ids = []
        
        for _ in range(max_len):
            tgt_mask = subsequent_mask(ys.size(1)).to(device)
            out = actual_model.decode(memory, src_mask, ys, tgt_mask)
            logits = actual_model.generator(out[:, -1])
            
            # Apply temperature
            if temperature != 1.0:
                logits = logits / temperature
            
            # Apply top-k filtering
            if top_k > 0:
                top_values, _ = torch.topk(logits, top_k)
                logits[logits < top_values[:, -1:]] = float('-inf')
            
            # Sample from the distribution
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            
            # Stop if EOS token
            if next_token == sp.eos_id():
                break
            
            # Avoid special tokens except sentinels
            if next_token in [sp.pad_id(), sp.unk_id()]:
                continue
                
            ys = torch.cat([ys, torch.tensor([[next_token]], device=device)], dim=1)
            generated_ids.append(next_token)
    
    # Decode and return
    return sp.decode(generated_ids).strip()


def generate_text_t5(model, sp, corrupted_input, max_len=100, device='cuda'):
    """
    Generate text using T5-style denoising (span corruption recovery).
    The model reconstructs the corrupted spans.
    """
    model.eval()
    
    # Handle DataParallel
    if isinstance(model, nn.DataParallel):
        actual_model = model.module
    else:
        actual_model = model
    
    with torch.no_grad():
        # Encode the corrupted input
        if isinstance(corrupted_input, str):
            tokens = sp.encode(corrupted_input, out_type=int)
            src = torch.tensor([sp.bos_id()] + tokens + [sp.eos_id()], device=device).unsqueeze(0)
        else:
            src = corrupted_input.to(device)
        
        src_mask = (src != sp.pad_id()).unsqueeze(-2)
        memory = actual_model.encode(src, src_mask)
        
        # Initialize with BOS token
        ys = torch.tensor([[sp.bos_id()]], device=device)
        
        for _ in range(max_len):
            tgt_mask = subsequent_mask(ys.size(1)).to(device)
            out = actual_model.decode(memory, src_mask, ys, tgt_mask)
            prob = actual_model.generator(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            next_word = next_word.item()
            
            if next_word == sp.eos_id():
                break
                
            ys = torch.cat([ys, torch.tensor([[next_word]], device=device)], dim=1)
        
        # Decode the output
        decoded_tokens = ys.cpu().numpy()[0].tolist()
        return sp.decode(decoded_tokens).strip()

print("✓ Inference functions defined successfully!")

## Evaluation Helper Functions

In [ ]:
import math
from tqdm import tqdm
import numpy as np
from difflib import SequenceMatcher

def calculate_perplexity(model, dataloader, device):
    """Calculate perplexity on a dataset."""
    model.eval()
    total_loss = 0.0
    total_tokens = 0
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction="sum")
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Perplexity", leave=False):
            if len(batch) == 3:
                src, tgt, _ = batch
            else:
                src, tgt = batch
            
            src, tgt = src.to(device), tgt.to(device)
            batch_obj = Batch(src, tgt, pad=0)
            
            out = model.forward(batch_obj.src, batch_obj.trg, batch_obj.src_mask, batch_obj.trg_mask)
            logits = model.generator(out)
            loss = criterion(logits.reshape(-1, logits.size(-1)), batch_obj.trg_y.reshape(-1))
            
            total_loss += loss.item()
            total_tokens += batch_obj.ntokens.item()
            
    avg_loss = total_loss / total_tokens if total_tokens > 0 else 0.0
    return math.exp(avg_loss) if avg_loss < 100 else float("inf")


def calculate_token_accuracy(pred_tokens, ref_tokens):
    """Calculate token-level accuracy."""
    if not ref_tokens:
        return 0.0
    matches = sum(1 for p, r in zip(pred_tokens, ref_tokens) if p == r)
    return matches / len(ref_tokens)


def calculate_word_overlap(pred, ref):
    """Calculate word overlap percentage."""
    pred_words = set(pred.split())
    ref_words = set(ref.split())
    if not ref_words:
        return 0.0
    intersection = pred_words & ref_words
    return len(intersection) / len(ref_words)


def calculate_char_similarity(pred, ref):
    """Calculate character-level similarity using sequence matcher."""
    if not ref:
        return 0.0
    matcher = SequenceMatcher(None, pred, ref)
    return matcher.ratio()


def calculate_length_ratio(pred, ref):
    """Calculate length ratio between prediction and reference."""
    if not ref:
        return 0.0
    return min(len(pred), len(ref)) / max(len(pred), len(ref))


def safe_bleu_score(predictions, references):
    """Calculate BLEU score with error handling."""
    try:
        import evaluate
        bleu = evaluate.load("bleu")
        result = bleu.compute(
            predictions=predictions, 
            references=[[r] for r in references],
            smooth=True
        )
        return result["bleu"] * 100
    except:
        return 0.0


def safe_rouge_score(predictions, references):
    """Calculate ROUGE-L score with error handling."""
    try:
        import evaluate
        rouge = evaluate.load("rouge")
        result = rouge.compute(predictions=predictions, references=references)
        return result["rougeL"] * 100
    except:
        return 0.0


def evaluate_model(model, dataloader, sp, device, max_samples=None):
    """Evaluate model and return comprehensive metrics."""
    model.eval()
    predictions, references, inputs = [], [], []
    sample_count = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating for Evaluation", leave=False):
            src, _, original_texts = batch
            src = src.to(device)
            
            for i in range(src.size(0)):
                if max_samples and sample_count >= max_samples:
                    break
                
                input_text = sp.decode(src[i].cpu().numpy().tolist())
                generated_text = generate_text_t5(model, sp, src[i].unsqueeze(0), device=device)
                
                inputs.append(input_text)
                predictions.append(generated_text)
                references.append(original_texts[i])
                
                sample_count += 1
            
            if max_samples and sample_count >= max_samples:
                break
    
    if not predictions:
        print("Warning: No valid predictions generated!")
        return None, [], [], []

    # Calculate metrics
    token_accuracies = []
    word_overlaps = []
    char_similarities = []
    length_ratios = []
    
    for pred, ref in zip(predictions, references):
        pred_tokens = sp.encode(pred, out_type=int)
        ref_tokens = sp.encode(ref, out_type=int)
        
        token_accuracies.append(calculate_token_accuracy(pred_tokens, ref_tokens))
        word_overlaps.append(calculate_word_overlap(pred, ref))
        char_similarities.append(calculate_char_similarity(pred, ref))
        length_ratios.append(calculate_length_ratio(pred, ref))
    
    bleu_score = safe_bleu_score(predictions, references)
    rouge_score = safe_rouge_score(predictions, references)
    
    metrics = {
        "bleu": bleu_score,
        "rougeL": rouge_score,
        "token_accuracy": np.mean(token_accuracies) * 100,
        "word_overlap": np.mean(word_overlaps) * 100,
        "char_similarity": np.mean(char_similarities) * 100,
        "length_ratio": np.mean(length_ratios) * 100,
    }
    
    return metrics, predictions, references, inputs

print("✓ Evaluation helper functions defined successfully!")

## Evaluate T5 Model (Span Corruption)

In [ ]:
print("\n" + "="*80)
print("EVALUATING T5 MODEL (SPAN CORRUPTION)")
print("="*80)

# Calculate perplexity for T5 model
val_perplexity_t5 = calculate_perplexity(model_t5, val_loader, device)
print(f"\n📊 T5 Model Validation Perplexity: {val_perplexity_t5:.4f}")

# Evaluate on test set
metrics_t5, predictions_t5, references_t5, inputs_t5 = evaluate_model(
    model_t5, test_loader, urdu_sp, device, max_samples=100
)

if metrics_t5:
    print("\n" + "="*80)
    print("T5 MODEL EVALUATION METRICS")
    print("="*80)
    
    print("\n📈 Primary Metrics (Custom):")
    print(f"  • Token Accuracy:       {metrics_t5['token_accuracy']:.2f}%")
    print(f"  • Word Overlap:         {metrics_t5['word_overlap']:.2f}%")
    print(f"  • Character Similarity: {metrics_t5['char_similarity']:.2f}%")
    print(f"  • Length Ratio:         {metrics_t5['length_ratio']:.2f}%")
    
    print("\n📊 Standard Metrics:")
    print(f"  • BLEU Score:           {metrics_t5['bleu']:.4f}")
    print(f"  • ROUGE-L Score:        {metrics_t5['rougeL']:.4f}")
    
    avg_custom_t5 = (metrics_t5['token_accuracy'] + metrics_t5['word_overlap'] + 
                     metrics_t5['char_similarity'] + metrics_t5['length_ratio']) / 4
    print(f"\n⭐ Average Custom Score: {avg_custom_t5:.2f}%")
    
    # Show examples
    print("\n" + "="*80)
    print("T5 MODEL - SAMPLE OUTPUTS")
    print("="*80)
    for idx in range(min(5, len(predictions_t5))):
        print(f"\n🔹 Example {idx + 1}")
        print(f"  Input (Corrupted):  {inputs_t5[idx][:80]}...")
        print(f"  Expected:           {references_t5[idx][:80]}...")
        print(f"  Model Output:       {predictions_t5[idx][:80]}...")
        
        char_sim = calculate_char_similarity(predictions_t5[idx], references_t5[idx])
        print(f"  Similarity:         {char_sim*100:.1f}%")

print("\n✓ T5 Model Evaluation Complete!")

## Evaluate Causal LM Model

In [ ]:
print("\n" + "="*80)
print("EVALUATING CAUSAL LM MODEL")
print("="*80)

# Calculate perplexity for Causal model
val_perplexity_causal = calculate_perplexity(model_causal, val_causal_loader, device)
print(f"\n📊 Causal Model Validation Perplexity: {val_perplexity_causal:.4f}")

# Test text generation with Causal model
test_prompts = [
    "میں اسکول",
    "یہ کتاب",
    "آج موسم",
    "میں نے کھانا",
    "وہ گھر",
    "تم کیسے",
    "میں ٹھیک"
]

print("\n" + "="*80)
print("CAUSAL LM MODEL - TEXT GENERATION EXAMPLES")
print("="*80)

for i, prompt in enumerate(test_prompts, 1):
    try:
        generated = generate_text_causal(model_causal, urdu_sp, prompt, max_len=30, temperature=0.7, top_k=40, device=device)
        print(f"\n🔹 Example {i}")
        print(f"  Prompt:      {prompt}")
        print(f"  Generated:   {prompt} {generated}")
    except Exception as e:
        print(f"\n🔹 Example {i}")
        print(f"  Prompt:      {prompt}")
        print(f"  Error:       {str(e)}")

# Calculate metrics on test set
print("\n" + "="*80)
print("CALCULATING COMPREHENSIVE METRICS FOR CAUSAL MODEL")
print("="*80)

# For causal model, we'll measure how well it continues given prompts
# We'll take the first few tokens as prompt and compare generation with the rest
causal_test_predictions = []
causal_test_references = []
causal_test_prompts = []

model_causal.eval()
with torch.no_grad():
    sample_count = 0
    for batch in tqdm(test_causal_loader, desc="Evaluating Causal LM"):
        src, tgt = batch
        src = src.to(device)
        
        for i in range(src.size(0)):
            if sample_count >= 100:
                break
            
            # Take first 5 tokens as prompt
            prompt_tokens = src[i][:5].cpu().numpy().tolist()
            prompt_text = urdu_sp.decode(prompt_tokens)
            
            # Full text as reference
            full_tokens = src[i].cpu().numpy().tolist()
            reference_text = urdu_sp.decode(full_tokens)
            
            # Generate continuation
            try:
                generated = generate_text_causal(model_causal, urdu_sp, prompt_text, max_len=30, temperature=0.7, device=device)
                full_generated = prompt_text + " " + generated
                
                causal_test_prompts.append(prompt_text)
                causal_test_predictions.append(full_generated)
                causal_test_references.append(reference_text)
                
                sample_count += 1
            except:
                continue
        
        if sample_count >= 100:
            break

# Calculate metrics
if causal_test_predictions:
    token_accuracies = []
    word_overlaps = []
    char_similarities = []
    length_ratios = []
    
    for pred, ref in zip(causal_test_predictions, causal_test_references):
        pred_tokens = urdu_sp.encode(pred, out_type=int)
        ref_tokens = urdu_sp.encode(ref, out_type=int)
        
        token_accuracies.append(calculate_token_accuracy(pred_tokens, ref_tokens))
        word_overlaps.append(calculate_word_overlap(pred, ref))
        char_similarities.append(calculate_char_similarity(pred, ref))
        length_ratios.append(calculate_length_ratio(pred, ref))
    
    bleu_score_causal = safe_bleu_score(causal_test_predictions, causal_test_references)
    rouge_score_causal = safe_rouge_score(causal_test_predictions, causal_test_references)
    
    metrics_causal = {
        "bleu": bleu_score_causal,
        "rougeL": rouge_score_causal,
        "token_accuracy": np.mean(token_accuracies) * 100,
        "word_overlap": np.mean(word_overlaps) * 100,
        "char_similarity": np.mean(char_similarities) * 100,
        "length_ratio": np.mean(length_ratios) * 100,
    }
    
    print("\n" + "="*80)
    print("CAUSAL LM MODEL EVALUATION METRICS")
    print("="*80)
    
    print("\n📈 Primary Metrics (Custom):")
    print(f"  • Token Accuracy:       {metrics_causal['token_accuracy']:.2f}%")
    print(f"  • Word Overlap:         {metrics_causal['word_overlap']:.2f}%")
    print(f"  • Character Similarity: {metrics_causal['char_similarity']:.2f}%")
    print(f"  • Length Ratio:         {metrics_causal['length_ratio']:.2f}%")
    
    print("\n📊 Standard Metrics:")
    print(f"  • BLEU Score:           {metrics_causal['bleu']:.4f}")
    print(f"  • ROUGE-L Score:        {metrics_causal['rougeL']:.4f}")
    
    avg_custom_causal = (metrics_causal['token_accuracy'] + metrics_causal['word_overlap'] + 
                         metrics_causal['char_similarity'] + metrics_causal['length_ratio']) / 4
    print(f"\n⭐ Average Custom Score: {avg_custom_causal:.2f}%")
    
    # Show examples
    print("\n" + "="*80)
    print("CAUSAL LM - SAMPLE CONTINUATIONS")
    print("="*80)
    for idx in range(min(5, len(causal_test_predictions))):
        print(f"\n🔹 Example {idx + 1}")
        print(f"  Prompt:      {causal_test_prompts[idx]}")
        print(f"  Reference:   {causal_test_references[idx][:80]}...")
        print(f"  Generated:   {causal_test_predictions[idx][:80]}...")
        
        char_sim = calculate_char_similarity(causal_test_predictions[idx], causal_test_references[idx])
        print(f"  Similarity:  {char_sim*100:.1f}%")

print("\n✓ Causal LM Evaluation Complete!")

# Final Model Packaging & Export

Package all models, tokenizer, configurations, and training history into a complete zip file for deployment and sharing.

In [ ]:
import zipfile
import json
from pathlib import Path
from datetime import datetime
import shutil

# Create export directory
export_dir = Path("/kaggle/working/model_export")
export_dir.mkdir(parents=True, exist_ok=True)

print("="*80)
print("PACKAGING MODELS FOR EXPORT")
print("="*80)

# 1. Save Model Configurations
print("\n📝 Saving model configurations...")

model_configs = {
    "model_architecture": {
        "src_vocab": len(urdu_sp),
        "tgt_vocab": len(urdu_sp),
        "N": 2,
        "d_model": 512,
        "d_ff": 2048,
        "h": 8,
        "dropout": 0.1,
    },
    "training_info": {
        "t5_epochs": num_epochs_t5,
        "t5_best_val_loss": best_val_loss_t5,
        "causal_epochs": num_epochs_causal,
        "causal_best_val_loss": best_val_loss_causal,
        "tokenizer_vocab_size": len(urdu_sp),
        "dataset_size": len(urdu_texts),
        "train_size": len(train_dataset),
        "val_size": len(val_dataset),
        "test_size": len(test_dataset_eval),
    },
    "evaluation_metrics": {
        "t5_model": metrics_t5 if metrics_t5 else {},
        "causal_model": metrics_causal if 'metrics_causal' in locals() else {},
        "t5_perplexity": val_perplexity_t5,
        "causal_perplexity": val_perplexity_causal if 'val_perplexity_causal' in locals() else None,
    },
    "export_info": {
        "export_date": datetime.now().isoformat(),
        "framework": "PyTorch",
        "python_version": "3.10+",
    }
}

config_path = export_dir / "model_config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(model_configs, f, indent=2, ensure_ascii=False)

print(f"✓ Configuration saved to {config_path}")

# 2. Save Training History
print("\n📊 Saving training history...")

training_history = {
    "t5_training": training_history_t5,
    "causal_training": training_history_causal,
}

history_path = export_dir / "training_history.json"
with open(history_path, 'w', encoding='utf-8') as f:
    json.dump(training_history, f, indent=2)

print(f"✓ Training history saved to {history_path}")

# 3. Save Model Bundles
print("\n💾 Saving model bundles...")

# T5 Model Bundle
t5_bundle = {
    "state_dict": {k: v.cpu() for k, v in base_model_t5.state_dict().items()},
    "config": model_configs["model_architecture"],
    "training_type": "t5_span_corruption",
    "best_val_loss": best_val_loss_t5,
    "exported_at": datetime.now().isoformat(),
}
t5_bundle_path = export_dir / "urdu_transformer_t5.pt"
torch.save(t5_bundle, t5_bundle_path)
print(f"✓ T5 model bundle saved to {t5_bundle_path}")

# Causal Model Bundle
causal_bundle = {
    "state_dict": {k: v.cpu() for k, v in base_model_causal.state_dict().items()},
    "config": model_configs["model_architecture"],
    "training_type": "causal_language_modeling",
    "best_val_loss": best_val_loss_causal,
    "pretrained_from": "t5_span_corruption",
    "exported_at": datetime.now().isoformat(),
}
causal_bundle_path = export_dir / "urdu_transformer_causal.pt"
torch.save(causal_bundle, causal_bundle_path)
print(f"✓ Causal model bundle saved to {causal_bundle_path}")

# 4. Copy Tokenizer Files
print("\n🔤 Copying tokenizer files...")

tokenizer_files = [
    "/kaggle/working/urdu_tokenizer.model",
    "/kaggle/working/urdu_tokenizer.vocab"
]

for tok_file in tokenizer_files:
    tok_path = Path(tok_file)
    if tok_path.exists():
        shutil.copy(tok_path, export_dir / tok_path.name)
        print(f"✓ Copied {tok_path.name}")

# 5. Create README
print("\n📄 Creating README...")

# Prepare conditional values safely
try:
    causal_perp_value = val_perplexity_causal
    causal_perp_str = f"{causal_perp_value:.4f}"
except (NameError, UnboundLocalError):
    causal_perp_str = 'N/A'

try:
    causal_score_value = avg_custom_causal
    causal_score_str = f"{causal_score_value:.2f}%"
except (NameError, UnboundLocalError):
    causal_score_str = 'N/A'

readme_content = f"""# Urdu Transformer Models - Export Package

**Export Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Package Contents

This package contains two trained transformer models for Urdu language processing:

### 1. T5-Style Span Corruption Model
- **File:** `urdu_transformer_t5.pt`
- **Training Method:** T5-style span corruption (denoising)
- **Best Validation Loss:** {best_val_loss_t5:.4f}
- **Epochs Trained:** {num_epochs_t5}

### 2. Causal Language Model
- **File:** `urdu_transformer_causal.pt`
- **Training Method:** Causal language modeling (GPT-style)
- **Best Validation Loss:** {best_val_loss_causal:.4f}
- **Epochs Trained:** {num_epochs_causal}
- **Pre-trained From:** T5 model

## Model Architecture

- **Layers (N):** 2 encoder + 2 decoder layers
- **Model Dimension (d_model):** 512
- **Feed-forward Dimension (d_ff):** 2048
- **Attention Heads (h):** 8
- **Dropout:** 0.1
- **Vocabulary Size:** {len(urdu_sp)}
- **Total Parameters:** ~{sum(p.numel() for p in model_t5.parameters()):,}

## Tokenizer

- **Type:** SentencePiece BPE
- **Vocabulary Size:** {len(urdu_sp)}
- **Files:** 
  - `urdu_tokenizer.model` (main model)
  - `urdu_tokenizer.vocab` (vocabulary)

## Dataset Information

- **Total Sentences:** {len(urdu_texts):,}
- **Training Split:** {len(train_dataset):,} samples (80%)
- **Validation Split:** {len(val_dataset):,} samples (10%)
- **Test Split:** {len(test_dataset_eval):,} samples (10%)

## Evaluation Metrics

### T5 Model
- **Perplexity:** {val_perplexity_t5:.4f}
- **Custom Score:** {avg_custom_t5:.2f}%

### Causal Model
- **Perplexity:** {causal_perp_str}
- **Custom Score:** {causal_score_str}

## Usage

### Loading the Models

```python
import torch
import sentencepiece as spm

# Load tokenizer
sp = spm.SentencePieceProcessor()
sp.load('urdu_tokenizer.model')

# Load T5 model
t5_checkpoint = torch.load('urdu_transformer_t5.pt', map_location='cpu')
# ... create model with config and load state_dict

# Load Causal model
causal_checkpoint = torch.load('urdu_transformer_causal.pt', map_location='cpu')
# ... create model with config and load state_dict
```

### Model Selection Guide

- **Use T5 Model for:**
  - Text denoising and correction
  - Filling in masked/corrupted text
  - Sequence-to-sequence tasks

- **Use Causal Model for:**
  - Text generation and continuation
  - Autocomplete functionality
  - Creative writing assistance

## Files Included

1. `urdu_transformer_t5.pt` - T5 model bundle
2. `urdu_transformer_causal.pt` - Causal LM model bundle
3. `urdu_tokenizer.model` - SentencePiece tokenizer
4. `urdu_tokenizer.vocab` - Tokenizer vocabulary
5. `model_config.json` - Complete configuration and metrics
6. `training_history.json` - Training progress history
7. `README.md` - This file

## Requirements

- Python 3.10+
- PyTorch 2.0+
- SentencePiece
- NumPy

## Authors

- Muhammad Abu Huraira (22F-3853)
- Shayan Zawar (22F-3410)

## License

See LICENSE file for details.

---

Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

readme_path = export_dir / "README.md"
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"✓ README saved to {readme_path}")

print("\n✅ All files prepared for export!")
print(f"\n📁 Export directory: {export_dir}")

In [ ]:
# Create the final ZIP file
print("\n" + "="*80)
print("CREATING ZIP ARCHIVE")
print("="*80)

zip_filename = "/kaggle/working/urdu_transformer_complete_export.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all files from export directory
    for file_path in export_dir.rglob('*'):
        if file_path.is_file():
            arcname = file_path.relative_to(export_dir.parent)
            zipf.write(file_path, arcname)
            print(f"  ✓ Added: {arcname}")

# Get file size
zip_size = Path(zip_filename).stat().st_size / (1024 * 1024)  # Size in MB

print("\n" + "="*80)
print("✅ EXPORT COMPLETE!")
print("="*80)
print(f"\n📦 ZIP File Created: {zip_filename}")
print(f"📊 File Size: {zip_size:.2f} MB")
print(f"\n📁 Package Contents:")
print(f"   • 2 Trained Models (T5 + Causal)")
print(f"   • Tokenizer (SentencePiece)")
print(f"   • Complete Configurations")
print(f"   • Training History")
print(f"   • Evaluation Metrics")
print(f"   • README Documentation")
print("\n🎉 Ready for download and deployment!")
print("="*80)

# Summary & Comparison

Compare the performance of both training approaches and summarize the complete pipeline.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("="*80)
print("FINAL SUMMARY & COMPARISON")
print("="*80)

# Create comparison table
print("\n📊 MODEL COMPARISON TABLE")
print("="*80)
print(f"{'Metric':<30} {'T5 Model':<20} {'Causal Model':<20}")
print("-"*80)

# Training info
print(f"{'Training Approach':<30} {'Span Corruption':<20} {'Causal LM':<20}")
print(f"{'Epochs Trained':<30} {num_epochs_t5:<20} {num_epochs_causal:<20}")
print(f"{'Best Val Loss':<30} {best_val_loss_t5:<20.4f} {best_val_loss_causal:<20.4f}")
print(f"{'Validation Perplexity':<30} {val_perplexity_t5:<20.4f} {val_perplexity_causal if 'val_perplexity_causal' in locals() else 0:<20.4f}")

# Metrics comparison
if metrics_t5 and 'metrics_causal' in locals():
    print("-"*80)
    print(f"{'Token Accuracy (%)':<30} {metrics_t5['token_accuracy']:<20.2f} {metrics_causal['token_accuracy']:<20.2f}")
    print(f"{'Word Overlap (%)':<30} {metrics_t5['word_overlap']:<20.2f} {metrics_causal['word_overlap']:<20.2f}")
    print(f"{'Char Similarity (%)':<30} {metrics_t5['char_similarity']:<20.2f} {metrics_causal['char_similarity']:<20.2f}")
    print(f"{'Length Ratio (%)':<30} {metrics_t5['length_ratio']:<20.2f} {metrics_causal['length_ratio']:<20.2f}")
    print("-"*80)
    print(f"{'BLEU Score':<30} {metrics_t5['bleu']:<20.4f} {metrics_causal['bleu']:<20.4f}")
    print(f"{'ROUGE-L Score':<30} {metrics_t5['rougeL']:<20.4f} {metrics_causal['rougeL']:<20.4f}")
    print("-"*80)
    print(f"{'Avg Custom Score (%)':<30} {avg_custom_t5:<20.2f} {avg_custom_causal:<20.2f}")

print("="*80)

# Plot training history
print("\n📈 Plotting training history...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Training History Comparison: T5 vs Causal LM', fontsize=16, fontweight='bold')

# T5 Loss
axes[0, 0].plot(training_history_t5['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(training_history_t5['val_loss'], label='Val Loss', marker='s')
axes[0, 0].set_title('T5 Model - Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# T5 Accuracy
axes[0, 1].plot(training_history_t5['train_acc'], label='Train Acc', marker='o', color='green')
axes[0, 1].plot(training_history_t5['val_acc'], label='Val Acc', marker='s', color='orange')
axes[0, 1].set_title('T5 Model - Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Causal Loss
axes[1, 0].plot(training_history_causal['train_loss'], label='Train Loss', marker='o')
axes[1, 0].plot(training_history_causal['val_loss'], label='Val Loss', marker='s')
axes[1, 0].set_title('Causal LM - Loss')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Causal Accuracy
axes[1, 1].plot(training_history_causal['train_acc'], label='Train Acc', marker='o', color='green')
axes[1, 1].plot(training_history_causal['val_acc'], label='Val Acc', marker='s', color='orange')
axes[1, 1].set_title('Causal LM - Accuracy')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/training_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Training history plot saved!")

# Create metrics comparison bar chart
if metrics_t5 and 'metrics_causal' in locals():
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    metrics_names = ['Token Accuracy', 'Word Overlap', 'Char Similarity', 'Length Ratio']
    t5_scores = [
        metrics_t5['token_accuracy'],
        metrics_t5['word_overlap'],
        metrics_t5['char_similarity'],
        metrics_t5['length_ratio']
    ]
    causal_scores = [
        metrics_causal['token_accuracy'],
        metrics_causal['word_overlap'],
        metrics_causal['char_similarity'],
        metrics_causal['length_ratio']
    ]
    
    x = range(len(metrics_names))
    width = 0.35
    
    bars1 = ax.bar([i - width/2 for i in x], t5_scores, width, label='T5 Model', color='skyblue')
    bars2 = ax.bar([i + width/2 for i in x], causal_scores, width, label='Causal LM', color='lightcoral')
    
    ax.set_xlabel('Metrics', fontweight='bold')
    ax.set_ylabel('Score (%)', fontweight='bold')
    ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/metrics_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Metrics comparison plot saved!")

# Summary
print("\n" + "="*80)
print("📋 COMPLETE PIPELINE SUMMARY")
print("="*80)
print("""
✅ Pipeline Completed Successfully!

Phase 1: T5-Style Span Corruption Training
  - Trained transformer with denoising objective
  - Model learns to reconstruct corrupted text spans
  - Best for text correction and infilling tasks

Phase 2: Causal Language Modeling
  - Fine-tuned T5 model with causal LM objective
  - Model learns to predict next tokens
  - Best for text generation and completion

Deliverables:
  ✓ Two fully trained models (T5 + Causal)
  ✓ SentencePiece tokenizer with 10K vocab
  ✓ Complete configuration files
  ✓ Training history and metrics
  ✓ Comprehensive evaluation results
  ✓ README documentation
  ✓ Complete ZIP package for deployment

All files are packaged and ready for download!
""")
print("="*80)